## Introduction

This notebook demonstrates how to visualize and analyze JWST MOS spectra, including how to:

* Launch [Mosviz](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html)
* [Load MOS data](https://jdaviz.readthedocs.io/en/latest/mosviz/import_data.html) from a notebook cell
* Adjust [display parameters](https://jdaviz.readthedocs.io/en/latest/cubeviz/displaycubes.html#display-settings)
* Select [spectral regions](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#spectral-regions) for further analysis 
* [Line analysis](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html#line-analysis): centroids, widths, and fluxes
* Load custom [line lists](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html#line-lists)


JWebbinar notebook created by Patrick Ogle at STScI

## Data

We use simulated NIRSpec MOS PRISM data created with the NIRSpec Instrument Performance Simulator (IPS).  This consists of 33 sources (primarily galaxies) plus background 
from 33 open slitlets. The dataset is not publicly released and will be replaced with another, better dataset when it becomes available.  Please do not distribute this version.

The simulated Level 2a data were run through the JWST Spec2 and Spec3 pipelines to create Level 2b and Level 3 (s2d and x1d) data products. Background subtraction was performed on the Level 3 data. The flux calibration for this dataset is inaccurate, pending on-orbit calibration.

## Imports
* [_jdaviz_](https://jdaviz.readthedocs.io/en/latest/) : [Mosviz](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html) data visualization tool
* [os](https://docs.python.org/3/library/os.html), [glob](https://docs.python.org/3/library/glob.html), [zipfile](https://docs.python.org/3/library/zipfile.html), and [astropy.utils.data.download_file](https://docs.astropy.org/en/stable/api/astropy.utils.data.download_file.html) for file management 
* [_astropy.units_]() and [astropy.table] for line lists


In [1]:
from jdaviz import Mosviz
import os
import glob
import zipfile
from astropy.utils.data import download_file

import numpy as np
import astropy.units as u
import astropy.table as t

## Visualize simulated NIRSpec MOS data

Execute the next cell to launch Mosviz, then follow the instructions for each task enumerated in the cell directly below the Cubeviz app.

In [2]:
mosviz = Mosviz()
mosviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

### UI Instructions:
#### Task 1:  Load the MOS data and view it
* Load the NIRSpec MOS spectra and image cutouts into Mosviz using the code cells below
* Click on the third row (not the checkbox) in the Mosviz table viewer to display data for one of the sources in the MOS dataset.
* Open the Display menu of the Image viewer. In the Layer tab, change the stretch to Logarithmic, 99 percentile.
* Try adjusting the stretch and colormap in the 2D spectrum viewer.

#### Task 2:  Toggle the slit viewer on and off
* Find the Slit Overlay Plugin by clicking the 'Lego' icon at upper right.
* Uncheck or check the box to turn the slit viewer off or on.

#### Task 3: Select a spectral region for further analysis
* Use the horizontal and vertical pan-zoom tools in the 1D spectrum viewer to zoom in on the spectral line at 2 microns.  Note that the 2D spectral region matches the selection in the 1D viewer.
* Use the spectral region selection tool in the 1D spectrum viewer to select the line at 2 microns (Subset 1).

#### Task 4: Measure line centroids using the Line Analysis plugin
* Find the Line Analysis Plugin by clicking the 'Lego' icon at upper right.
* Select a dataset with the Data dropdown (e.g. Subset1).
* Several line properties are automatically computed (flux, equivalent width, Gaussian sigma, Gaussian FWHM, and Centroid) using specutils functionality. Note that the line flux, Gaussian sigma, FWHM, and centroid require continuum subtraction or normalization, and will not be correct if there is a significant continuum component. The equivalent width, on the other hand, requires the continuum to be normalized to unity to give a correct answer. 
* The Halpha emission line complex is at 1.92 microns (redshift z = 1.93) 
* Now determine the centroid wavelength of the line at ~1.5 microns.  What is the line ID?

#### Task 5: Identify spectral lines using the Line Lists plugin
* Open the Line Lists plugin to create a custom line list: OIII5007 (1.467 um), Halpha (1.923 um)
* Click on the 'Custom' tab and enter the first line name, wavelength, and wavelength unit, and click 'Add Line'
* In the same place, enter the values for the second line and 'Add Line'
* Click through the other spectra to see if those same lines appear (Hint: it is one of the first 5)

#### Task 6: Load a line list from the notebook and display them in the spectrum viewer
* Run the cell below to load a custom line list programmatically
* Open the Line Lists plugin
* Change the color of the 'Galaxy3' line list to distinguish from the 'Custom' line list
* Click on Hide/Show All in the Galaxy3 line list to toggle lines on and off
* Check/uncheck individual lines to show only the ones you want


In [3]:
#Redshifted optical emission lines

line = ['OII', 'Hbeta', 'OIIIa', 'OIIIb', 'Halpha', 'SII', 'Palpha']
wavel = [.3727, .4861, .4959, .5007, .6563, .6724, 1.8756]
z =1.93

wave = np.array(wavel)*(1+z)
print(line)
print(wave)

#Create an astropy table 
lt = t.QTable()
lt['linename'] = line
lt['rest'] = wavel*u.um
lt['redshift'] = u.Quantity(z)
lt['listname'] = "Galaxy3"
print(lt)

#Load the line list table into the Line Lists plugin
mosviz.specviz.load_line_list(lt)

['OII', 'Hbeta', 'OIIIa', 'OIIIb', 'Halpha', 'SII', 'Palpha']
[1.092011 1.424273 1.452987 1.467051 1.922959 1.970132 5.495508]
linename  rest  redshift listname
           um                    
-------- ------ -------- --------
     OII 0.3727     1.93  Galaxy3
   Hbeta 0.4861     1.93  Galaxy3
   OIIIa 0.4959     1.93  Galaxy3
   OIIIb 0.5007     1.93  Galaxy3
  Halpha 0.6563     1.93  Galaxy3
     SII 0.6724     1.93  Galaxy3
  Palpha 1.8756     1.93  Galaxy3


### Download the data

In [4]:
# From the Jwebbinar platform
#!ls /home/shared/preloaded-fits/nirspec_files/mos_prism
#data_dir = '/home/shared/preloaded-fits/nirspec_files/mos_prism'

#From Box
#Notes: 
#1) This requires the directory ./mosdata to already exist
#2) This dataset is not publically available yet (ask Patrick Ogle)
data_dir ='./mosdata'
ziplink = 'https://stsci.box.com/shared/static/xddvu98qa5k2ggjz3ru4az37t63anr1g.zip'
zipfilename = 'nirspec_mosdata_prism.zip'
print(zipfilename)
if not os.path.isfile(os.path.join(data_dir, zipfilename)):
    print('Downloading {}...'.format(zipfilename))
    demo_file = download_file(ziplink, cache=True)
    # Make a symbolic link using a local name for convenience
    print(os.path.join(data_dir, zipfilename))
    os.symlink(demo_file, os.path.join(data_dir, zipfilename))
else:
    print('{} already exists, skipping download...'.format(zipfilename))
zf = zipfile.ZipFile(data_dir+'/'+zipfilename, 'r')
zf.extractall(data_dir)


nirspec_mosdata_prism.zip
./mosdata/nirspec_mosdata_prism.zip


### Create the lists of data products
Skip this cell if you want Mosviz to load by directory instead of file list

In [ ]:
# Create the lists of data products
slit_id = []
spectra_1d = []
spectra_2d = []
cutouts = []
for file_path in glob.glob(str(data_dir+'/*')):
    #print(file_path)
    if ('x1d' in file_path):
        spectra_1d.append(file_path)
        slitid = file_path[17:22]
        slit_id.append(slitid)
    elif ('s2d' in file_path):
        spectra_2d.append(file_path)
    elif ('cutout' in file_path):
        cutouts.append(file_path)     

#Sort the slit ids and filenames
slit_id_sorted = sorted (slit_id)    
spectra_1d_sorted = sorted(spectra_1d)
spectra_2d_sorted = sorted(spectra_2d)
cutouts_sorted = sorted(cutouts)
  
#Print out the matched filenames for each slit
for s_id, f_1d, f_2d, f_cut in zip(slit_id_sorted, spectra_1d_sorted, spectra_2d_sorted, cutouts_sorted): 

    print(s_id, f_1d[10:], '...'+f_2d[44:], f_cut[10:])

## Load the data into Mosviz

In [5]:

#Old method:
#mosviz.load_data(spectra_1d_sorted, spectra_2d_sorted, images=cutouts_sorted)

#Alternative old method, to load one type of data at a time: 
#mosviz.load_metadata(cutouts_sorted)            
#mosviz.load_1d_spectra(spectra_1d_sorted)  
#mosviz.load_2d_spectra(spectra_2d_sorted) 
#mosviz.load_images(cutouts_sorted)

#New method: Mosviz convenience function to to load data from directory
#(Don't need to run the previous cell)
data_dir ='./mosdata'
mosviz.load_data(directory=data_dir, instrument="nirspec")
